In [ ]:
from keras.models import Sequential, Model, load_model
from keras.layers import Conv2D, MaxPooling2D, Activation, Dropout, Flatten, Dense, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras import backend as K
from keras.applications.xception import Xception
from keras.callbacks import ModelCheckpoint, EarlyStopping

import os
import numpy as np

# K.set_image_dim_ordering('th')
K.set_image_dim_ordering('tf')

path_to_data = '/home/cody/code/pokemon_classifier/dataset'
path_to_validation_data = '/home/cody/code/pokemon_classifier/validation_dataset'
model_path = '/home/cody/code/pokemon_classifier/models'
batch_size = 30

### Train Model

In [ ]:
base_model = Xception(input_shape=(299, 299, 3), include_top=False, weights='imagenet')

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True
)

train_generator = train_datagen.flow_from_directory(
    path_to_data,  # this is the target directory
    target_size=(299, 299),  # all images will be resized to 299x299
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)


In [ ]:
# Get the indices, we'll use these when testing (associate output with pokemon name)
class_indices = train_generator.class_indices

import json
with open(model_path + '/class_indices.json', 'w') as outfile:
    json.dump(class_indices, outfile)

In [ ]:
validation_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

validation_generator = validation_datagen.flow_from_directory(
    path_to_validation_data,
    target_size=(299, 299),
    batch_size=batch_size,
    shuffle=True,
    class_mode='categorical'
)

In [ ]:
# Top Model Block
x = base_model.output
x = GlobalAveragePooling2D()(x)
predictions = Dense(151, activation='softmax')(x)

# add your top layer block to your base model
model = Model(base_model.input, predictions)

print(model.summary())

for layer in base_model.layers:
    layer.trainable = False

model.compile(optimizer='nadam', loss='categorical_crossentropy', metrics=['accuracy'])

top_weights_path = os.path.join(os.path.abspath(model_path), 'top_model_weights.h5')
callbacks_list = [
    ModelCheckpoint(top_weights_path, monitor='acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='val_acc', patience=5, verbose=0)
]

# fine-tune the model first only top layer
model.fit_generator(
        train_generator,
        steps_per_epoch = train_generator.samples // batch_size,
        use_multiprocessing=True,
        epochs=16,
        # epochs=100,
        validation_data=validation_generator,
        validation_steps = validation_generator.samples // batch_size,
        callbacks=callbacks_list
)

In [ ]:
model.load_weights(top_weights_path)

In [ ]:
based_model_last_block_layer_number = 126

for layer in model.layers[:based_model_last_block_layer_number]:
    layer.trainable = False
for layer in model.layers[based_model_last_block_layer_number:]:
    layer.trainable = True

model.compile(optimizer='nadam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])

final_weights_path = os.path.join(os.path.abspath(model_path), 'model_weights.h5')
callbacks_list = [
    ModelCheckpoint(final_weights_path, monitor='acc', verbose=1, save_best_only=True),
    EarlyStopping(monitor='loss', patience=5, verbose=0)
]

model.fit_generator(
    train_generator,
    steps_per_epoch = train_generator.samples // batch_size,
    use_multiprocessing=True,
    shuffle=True,
    epochs=16,
    validation_data=validation_generator,
    validation_steps = validation_generator.samples // batch_size,
    callbacks=callbacks_list
)

### Save Model

In [ ]:
# save model as architecture and weights
model_json = model.to_json()
with open(os.path.join(os.path.abspath(model_path), 'model.json'), 'w') as json_file:
    json_file.write(model_json)

In [ ]:
# save whole thing
model.save('models/my_model.h5')

### Load Model

In [ ]:
# load model as architecture and weights

from keras.models import model_from_json

json_file = open(model_path + "/model.json", 'r')
loaded_model_json = json_file.read()
json_file.close()

test_model = model_from_json(loaded_model_json)

test_model.load_weights(model_path + '/model_weights.h5')

In [ ]:
# load whole thing
test_model = load_model('models/my_model.h5')

### Test Model

In [ ]:

img = load_img(path_to_data + '/Starmie/b05d5dc1b3364dc8ac42f593ad3a479f.jpg', target_size=(299,299))
x = img_to_array(img)
x = np.expand_dims(x, axis=0)

pred = model.predict(x)
np.argmax(pred, axis=1)